<a href="https://colab.research.google.com/github/Tanguyvans/StGeorge/blob/main/Managing_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install -q kaggle
!pip install ImageHash

import urllib
import os
import pandas as pd
import imagehash
from tqdm import tqdm
from PIL import Image
import random
import shutil

from traitlets.traitlets import default

from google.colab import files
from google.colab import drive

from os import listdir
from os.path import isfile, join

In [ ]:
drive.mount('/content/gdrive')

!mkdir /content/gdrive/MyDrive/george_ds
!mkdir /content/gdrive/MyDrive/george_ds/george
!mkdir /content/gdrive/MyDrive/george_ds/no_george

# Adding new data to the dataset from Kaggle

In [ ]:
add_data = True

def adding_data(source, target): 
  files = [f for f in listdir(source) if isfile(join(source, f))]
  targetfiles = [f for f in listdir(target) if isfile(join(target, f))]

  for file in files: 
    if file not in target: 
      dirpath = source+file
      !cp {dirpath} {targetpath}

if add_data: 
  files.upload()
  !rm -r ~/.kaggle
  !mkdir ~/.kaggle
  !mv ./kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json
  !kaggle datasets list
  !kaggle datasets download -d gobsan/st-george-or-not
  !unzip st-george-or-not.zip

In [12]:
if add_data: 
  # For the george data
  sourcepath = '/content/data/g/'
  targetpath = '/content/gdrive/MyDrive/george_ds/george/'
  adding_data(sourcepath, targetpath)

  # for no_george data 
  sourcepath = '/content/data/n_g/'
  targetpath = '/content/gdrive/MyDrive/george_ds/no_george/'
  adding_data(sourcepath, targetpath)


# Filling Dataset with csv file

In [14]:
get_images = True

source_path = '/content/gdrive/MyDrive/test_assignment_cv/'
dataset_path = '/content/gdrive/MyDrive/george_ds/'

if get_images: 

  # making directories
  if os.path.exists(dataset_path) == False:
    os.mkdir(dataset_path+'george')
    os.mkdir(dataset_path+'no_george')

  # reading csv files
  george = pd.read_csv(source_path+'georges.csv', header = None)
  non_george = pd.read_csv(source_path+'non_georges.csv', header = None)

  # george elements
  for index, row in george.iterrows(): 
    urllib.request.urlretrieve(row[0], dataset_path+'george/'+str(index)+'.jpg')

  # no george elements
  for index, row in non_george.iterrows():
    urllib.request.urlretrieve(row[0], dataset_path+'no_george/'+str(index)+'.jpg')

# Checking for duplicates

In [ ]:
check_duplicates = True

if check_duplicates: 
  classes = ['george/', 'no_george/']

  # creating a dict with the hash of the images. If hash is allreading in dict than it is a duplicate
  hash_dict = {}
  for classe in classes:
    path = dataset_path + classe
    for image in tqdm(os.listdir(path)):
      path_image = path + image

      # if it is not a jpg file, the file should be deleted
      if 'jpg' not in image: 
        os.remove(path_image)
        continue
      hash = imagehash.dhash(Image.open(path_image))
      if hash not in hash_dict:
        hash_dict[hash] = path_image

      else:
        #print(f'{hash_dict[hash]} is equal to {path_image}')
        os.remove(path_image)
        print(f'{path_image} removed')

# Creating test dataset

In [16]:
!mkdir /content/gdrive/MyDrive/george_test_ds
!mkdir /content/gdrive/MyDrive/george_test_ds/george
!mkdir /content/gdrive/MyDrive/george_test_ds/no_george

In [17]:
source = '/content/gdrive/MyDrive/george_ds/'
dest = '/content/gdrive/MyDrive/george_test_ds/'
files = os.listdir(source+'george/')
no_of_files = len(files) // 10
for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source+'george/', file_name), dest+'george/')

files = os.listdir(source+'no_george/')
no_of_files = len(files) // 10
for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source+'no_george/', file_name), dest+'no_george/')